In [1]:
import numpy as np; np.random.seed(123456)
import pandas as pd
DATA_PATH = "/root/mats/CharacterTraining/data"

In [2]:
large = False
model = "gemma"

In [3]:
path = f"{DATA_PATH}/critiques/outputs/{model}/rephrased"
path += "_large" if large else ""
path += ".jsonl"
outputs = pd.read_json(path, orient="records", lines=True)
path = f"{DATA_PATH}/critiques/inputs/{model}/rephrased"
path += "_large" if large else ""
path += ".jsonl"
inputs = pd.read_json(path, orient="records", lines=True)
inputs["response"] = outputs["output"]
inputs["chosen"] = inputs.apply(
    lambda row: [
        {
            "role": "user",
            "content": row["question"][0]["content"]
        },
        {
            "role": "assistant",
            "content": row["response"]
        }
    ], axis=1
)
inputs["rejected"] = inputs.apply(
    lambda row: [
        {
            "role": "user",
            "content": row["question"][0]["content"]
        },
        {
            "role": "assistant",
            "content": row["question"][1]["content"]
        }
    ], axis=1
)
inputs = inputs[["chosen", "rejected"]]
indices = np.random.permutation(len(inputs))
start, end = len(inputs)//2, int(len(inputs)*0.95)
inputs = inputs.iloc[indices].iloc[start:end].reset_index(drop=True)
inputs.to_json(f"{DATA_PATH}/{model}_dpo.jsonl", orient="records", lines=True)